## Ony Novianti
#### 2041720029 (TI-3A)


In [ ]:
# Sumber : https://github.com/cloudxlab/bigdata/blob/master/spark/examples/mllib/ml-recommender.scala

In [ ]:
# Connect Google Drive Untuk Ambil Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=1273896f0f6f5fdbff43ca5a3d6ad0f26387baeaaf2f33dfe2db0c73c0db1786
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
# Import Library
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Movie Lens").getOrCreate()
sc = spark.sparkContext

In [ ]:
raw = sc.textFile("/content/drive/MyDrive/TI-3A/ml-1m/ratings.dat")
mydata = [(2, 1), (3, 3), (4, 2)]
mydatardd = sc.parallelize(mydata).map(lambda x: Rating(0, x[0], x[1]))

In [ ]:
class Rating:
    def __init__(self, userId, movieId, rating):
        self.userId = userId
        self.movieId = movieId
        self.rating = rating

# Parse String Menjadi Objek Rating
def parseRating(str):
    fields = str.split("::")
    assert len(fields) == 4
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]))



In [ ]:
ratings = raw.map(parseRating)
ratings_df = ratings.toDF(["userId", "movieId", "rating"])

# Convert the additional data to a DataFrame
mydata_df = mydatardd.toDF(["userId", "rating"])

# Combine the ratings and additional data
totalratings_df = ratings_df.union(mydata_df)

# Build the recommendation model using ALS
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(totalratings_df)

# Generate product recommendations for user ID 1
products = model.recommendForUserSubset(spark.createDataFrame([(1,)]).toDF("userId"), 10)
# Load data from movies, join it, and display the names ordered by ratings

In [ ]:
products.show()


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{1, 3740.3484}, ...|
+------+--------------------+

